In [1]:
import spacy

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
training_data = [
    ("I wish to buy a laptop.", {"entities":[(10, 12, "ACTION"), (16, 21, "PRODUCT")]}),
    ("What do you reccomend for a student.", {"entities":[(13, 21, "ACTION"), (29, 35, "TYPE")]}),
    ("What is the best device for a working professional.", {"entities":[(18, 23, "PRODUCT"), (31, 50, "TYPE")]}),
    ("I wish to buy something in the 50000 range.", {"entities":[(11, 13, "ACTION"), (32, 36, "PRICE"), (38, 42, "PRICE_TYPE")]}),
    ("What is in stock for MSI?", {"entities":[(22, 24, "BRAND")]}),
    ("I want to buy an i7 laptop.", {"entities":[(11, 13, "ACTION"), (18, 19, "CONFIG")]}),
    ("I want to buy an ryzen 5 laptop.", {"entities":[(11, 13, "ACTION"), (18, 24, "CONFIG")]}),
    ("I want to buy an i7 laptop under 50k", {"entities":[(11, 13, "ACTION"), (18, 19, "CONFIG"), (28, 32, "PRICE_TYPE"), (34, 36, "PRICE")]}),
    ("I want to buy an ryzen 5 laptop under 50k", {"entities":[(11, 13, "ACTION"), (18, 24, "CONFIG"), (33, 37, "PRICE_TYPE"), (39, 41, "PRICE")]}),
    ("What is in stock with a GPU.", {"entities":[(25, 27, "COMPONENT")]}),
    ("What is in stock for Dell?", {"entities":[(22, 25, "BRAND")]}),
    ("What is in stock for HP?", {"entities":[(22, 23, "BRAND")]}),
    ("What is in stock for Asus over 70000.", {"entities":[(22, 26, "BRAND"), (28, 31, "PRICE_TYPE"), (33, 37, "PRICE")]})
]

In [4]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [5]:
ner = nlp.get_pipe("ner")

In [6]:
for _, annotations in training_data:
    for ent in annotations["entities"]:
        ner.add_label(ent[2])

In [7]:
disable_pipe = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

In [11]:
from spacy.util import minibatch, compounding
from pathlib import Path
import random

In [9]:
from helper import Verbose
verbose = Verbose()

In [10]:
epochs = 100

In [13]:
with nlp.disable_pipe(*disable_pipe):
    optimizer = nlp.resume_training()
    for i in range(epochs):
        random.shuffle(training_data)
        losses = {}

        batches = minibatch(training_data, size=compounding(1.0, 4.0, 1.001))
        for batch in batches:
            text, annotation = zip(*batch)
            nlp.update(
                text,
                annotation,
                drop=0.5,
                losses=losses,
                sgd=optimizer
            )
        
        verbose.make_verbose(losses, i+1, epochs)

ValueError: [E989] `nlp.update()` was called with two positional arguments. This may be due to a backwards-incompatible change to the format of the training data in spaCy 3.0 onwards. The 'update' function should now be called with a batch of Example objects, instead of `(text, annotation)` tuples. 

In [14]:
type(nlp)

spacy.lang.en.English